In [1]:
import pickle
import shutil
import os

import xlsxwriter
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Charge les référentiels
cf referentiels.py

In [2]:
ROME_df = pd.read_csv('../referentiels/referentiel_ROME/20150921_arboprincipale28427_ROME.csv', index_col=0, sep='|', dtype=str)
OGR_df = pd.read_csv('../referentiels/referentiel_OGR/20150921_arboprincipale28427_OGR.csv', sep='|', dtype=str).set_index('OGR')
NAF_df = pd.read_csv('../referentiels/referentiel_NAF/naf2008_liste_n5_nouveau_header.csv', sep='|', encoding="utf-8").set_index(['NAF'])

In [3]:
tranches_effectif = ['00', '01', '02', '03', '11', '12', '21', '22', '31', '32', '41', '42', '51', '52', '53', 'NN']
seuils_tranches_effectif = [0, 1, 3, 6, 10, 20, 50, 100, 200, 250, 500, 1000, 2000, 5000, 10000]

# Charge les données

In [4]:
with open('array_ROME1_ETT.pickle', 'rb') as f:
    ROME1_ETT = pickle.load(f)
with open('array_ROME1_fusion.pickle', 'rb') as f:
    ROME1_fusion = pickle.load(f)
with open('array_ROME1_CDI_CDD.pickle', 'rb') as f:
    ROME1_CDI_CDD = pickle.load(f)
with open('array_ROME2_ETT.pickle', 'rb') as f:
    ROME2_ETT = pickle.load(f)
with open('array_ROME2_fusion.pickle', 'rb') as f:
    ROME2_fusion = pickle.load(f)
with open('array_ROME2_CDI_CDD.pickle', 'rb') as f:
    ROME2_CDI_CDD = pickle.load(f)
with open('array_offres.pickle', 'rb') as f:
    offres = pickle.load(f)
with open('array_offres_OGR.pickle', 'rb') as f:
    offres_OGR = pickle.load(f)


In [5]:
len(ROME_df), len(OGR_df)

(531, 10877)

# Crée un XLSX

In [6]:
target_dir = '/data/projects/pole-emploi/ROME_NAF/tableaux'

In [7]:
offres_par_NAF = offres.sum(1)
embauches_par_NAF = ROME1_fusion.sum(2).sum(1)
suggestion_array = np.zeros((len(NAF_df), len(ROME_df)))

def write_worksheet(worksheet, ROME, formats, suggestion_array):
    ROME_label = ROME_df.loc[ROME].label
    ROME_index = ROME_df.index.get_loc(ROME)
    
    row_cursor = 0
    
    worksheet.set_column(1, 1, 65)  # Labels

    # ROME
    
    worksheet.write(row_cursor, 0, ROME)
    worksheet.write(row_cursor, 1, ROME_label)
    row_cursor += 2
    
    nb_embauches = ROME1_fusion[:, ROME_index, :].sum()
    worksheet.write(row_cursor, 0, nb_embauches)
    worksheet.write(row_cursor, 1, 'embauches pour ce ROME (ROME principal)')
    row_cursor += 1
    
    nb_offres = offres[:, ROME_index].sum()
    worksheet.write(row_cursor, 0, nb_offres)
    worksheet.write(row_cursor, 1, "offres d'emploi pour ce ROME")
    row_cursor += 2

    
    # Codes OGR
    
    worksheet.write(row_cursor, 0, "Code OGR")
    worksheet.write(row_cursor, 1, "Label OGR")
    worksheet.write(row_cursor, 2, "Offres >= 30j (tous NAF)")    
    row_cursor += 1

    OGRs = list(OGR_df[OGR_df.ROME==ROME].index)
    nb_OGR = len(OGRs)
    poids_OGR = np.array([offres_OGR[:, OGR_df.index.get_loc(OGR)].sum() for OGR in OGRs])
    sorted_OGR_codes = [OGRs[i] for i in (-poids_OGR).argsort()]
    for i in range(nb_OGR):
        OGR = sorted_OGR_codes[i]
        OGR_label = OGR_df.loc[OGR].label
        OGR_index = OGR_df.index.get_loc(OGR)

        worksheet.write(row_cursor + i, 0, OGR)
        worksheet.write(row_cursor + i, 1, OGR_label)
        worksheet.write(row_cursor + i, 2, offres_OGR[:, OGR_index].sum())
    row_cursor += nb_OGR + 1

    # Tri
    
    poids_NAF = ROME1_fusion[:, ROME_index, :].sum(1)
    sorted_NAF_indexes = (-poids_NAF).argsort()  # [:20]
    sorted_NAF_codes = [NAF_df.iloc[NAF_index].name for NAF_index in sorted_NAF_indexes]
    sorted_NAF_labels = [NAF_df.loc[NAF_code].label for NAF_code in sorted_NAF_codes]
    
    # Données
    
    embauches_ROME1 = np.array([ROME1_fusion[NAF_index, ROME_index, :].sum() for NAF_index in sorted_NAF_indexes])
    part_embauches_ROME1 = embauches_ROME1 / nb_embauches
    embauches_ROME2 = np.array([ROME2_fusion[NAF_index, ROME_index, :].sum() for NAF_index in sorted_NAF_indexes])
    quotient_ROME = embauches_ROME1 / (embauches_ROME1 + embauches_ROME2)
    offres_column = np.array([offres[NAF_index, ROME_index] for NAF_index in sorted_NAF_indexes])
    part_offres = offres_column / nb_offres
    embauches_NAF = np.array([embauches_par_NAF[NAF_index] for NAF_index in sorted_NAF_indexes])
    part_embauches_NAF = embauches_ROME1/embauches_NAF
    offres_NAF = np.array([offres_par_NAF[NAF_index] for NAF_index in sorted_NAF_indexes])
    part_offres_NAF = offres_column/offres_NAF
    
    # Couleurs
    
    def couleur_colonne(values):
        return [formats['green'] if b else formats['red'] for b in values]
    
    def couleur_colonne_pourcent(values):
        return [formats['pourcent_green'] if b else formats['pourcent_red'] for b in values]
    
    seuil_embauches_ROME1 = (embauches_ROME1 >= 10)
    couleur_embauches_ROME1 = couleur_colonne(seuil_embauches_ROME1)

    seuil_part_embauches_ROME1 = (part_embauches_ROME1 >= 0.01)
    couleur_part_embauches_ROME1 = couleur_colonne_pourcent(seuil_part_embauches_ROME1)
    
    seuil_quotient_ROME = (quotient_ROME >= 0.66)
    couleur_quotient_ROME = couleur_colonne_pourcent(seuil_quotient_ROME)
    
    seuil_offres = (offres_column >= 10)
    couleur_offres = couleur_colonne(seuil_offres)
    
    seuil_part_offres = (part_offres >= 0.01)
    couleur_part_offres = couleur_colonne_pourcent(seuil_part_offres)
    
    seuil_part_embauches_NAF = (part_embauches_NAF >= 0.01)
    couleur_part_embauches_NAF = couleur_colonne_pourcent(seuil_part_embauches_NAF)
    
    seuil_part_offres_NAF = (part_offres_NAF >= 0.01)
    couleur_part_offres_NAF = couleur_colonne_pourcent(seuil_part_offres_NAF)
    
    # Suggestion
    suggestion = (
        seuil_embauches_ROME1.astype(int) + seuil_part_embauches_ROME1 + seuil_quotient_ROME +
        seuil_offres + seuil_part_offres + seuil_part_embauches_NAF + seuil_part_offres_NAF) > 3.5
    suggestion = suggestion.astype(int)
    for i, NAF_index in enumerate(sorted_NAF_indexes):
        suggestion_array[NAF_index, ROME_index] = suggestion[i]
    
        
    # Ecriture    
    
    header_col = 0
    def write_column(title, comment, values=None, column_format=None):
        nonlocal header_col
        worksheet.write(row_cursor, header_col, title, formats['header'])
        worksheet.write_comment(row_cursor, header_col, comment)
        if values is not None:
            if isinstance(column_format, list):
                for i in range(len(NAF_df)):
                    worksheet.write(row_cursor + 1 + i, header_col, values[i], column_format[i])
            else:
                worksheet.write_column(row_cursor + 1, header_col, values, column_format)
                    
        header_col += 1

    write_column("Code NAF", "Les codes NAF (de septembre 2015) sont triés par ordre décroissant du nombre d'embauches (en ROME principal)",
        sorted_NAF_codes)
    write_column("Label NAF", "",
        sorted_NAF_labels)
    write_column("Décision", "Laisser vide pour suivre la suggestion, 1 pour un couple ROME/NAF à garder, 0 pour un couple ROME/NAF à ignorer",
        )
    write_column("Suggestion", "Plus de colonnes vertes que rouges",
        suggestion)
    write_column("R1", "Contrats >= 30j (ROME principal)",
        embauches_ROME1, couleur_embauches_ROME1)
    write_column("%R1", "Part des embauches pour ce ROME (sur le ROME principal uniquement)",
        part_embauches_ROME1, couleur_part_embauches_ROME1)
    write_column("R2", "Contrats >= 30j (ROME secondaire)",
        embauches_ROME2)
    write_column("Part R1", "Part des embauches pour lequel le code ROME est principal et non secondaire (= R1/R1+R2)",
        quotient_ROME, couleur_quotient_ROME)
    write_column("OF", "Offres >= 30j",
        offres_column, couleur_offres)
    write_column("%OF", "Part des offres pour ce ROME",
        part_offres, couleur_part_offres)
    write_column("EmbauchesNAF", "Embauches pour ce NAF (tous ROME confondus)",
        embauches_NAF)
    write_column("%EmbauchesNAF", "Part des embauches de ce NAF (= R1/EmbauchesNAF)",
        part_embauches_NAF, couleur_part_embauches_NAF)
    write_column("OffresNAF", "Offres pour ce NAF (tous ROME confondus)",
        offres_NAF)
    write_column("%OffresNAF", "Part des offres de ce NAF (= OF/OffresNAF)",
        part_offres_NAF, couleur_part_offres_NAF)
    write_column("Commentaire", "",
        )


In [8]:
shutil.rmtree(target_dir, ignore_errors=True)
os.mkdir(target_dir)

familles = sorted(set(ROME_df.famille))
for famille in familles:
    ROME_famille = ROME_df[ROME_df.famille==famille]
    famille_label = ROME_famille.iloc[0].famille_label
    
    famille_dir_name = '{} - {}'.format(famille, famille_label)
    os.mkdir(os.path.join(target_dir, famille_dir_name))

    domaines = sorted(set(ROME_famille.domaine))
    for domaine in domaines:
        ROME_domaine = ROME_famille[ROME_famille.domaine==domaine]
        domaine_label = ROME_domaine.iloc[0].domaine_label

        domaine_filename = '{} - {}.xlsx'.format(domaine, domaine_label)
        domaine_path = os.path.join(target_dir, famille_dir_name, domaine_filename)
        workbook = xlsxwriter.Workbook(domaine_path, {'nan_inf_to_errors': True})

        format_pourcent = workbook.add_format()
        format_pourcent.set_num_format(9)  # Magic number
        format_pourcent_red = workbook.add_format()
        format_pourcent_red.set_num_format(9)  # Magic number
        format_pourcent_red.set_bg_color('red')
        format_pourcent_green = workbook.add_format()
        format_pourcent_green.set_num_format(9)  # Magic number
        format_pourcent_green.set_bg_color('green')
        format_red = workbook.add_format({'bg_color': 'red'})
        format_green = workbook.add_format({'bg_color': 'green'})
        format_header = workbook.add_format({'bold': True})
        
        ROMEs = sorted(set(ROME_domaine.index))
        for ROME in ROMEs:
            worksheet = workbook.add_worksheet(ROME)
            
            write_worksheet(worksheet, ROME, {
                'pourcent': format_pourcent,
                'pourcent_red': format_pourcent_red,
                'pourcent_green': format_pourcent_green,
                'red': format_red,
                'green': format_green,
                'header': format_header,
            }, suggestion_array)

        workbook.close()



/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:63: RuntimeWarning: invalid value encountered in true_divide
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:67: RuntimeWarning: invalid value encountered in true_divide
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:69: RuntimeWarning: invalid value encountered in true_divide
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:85: RuntimeWarning: invalid value encountered in greater_equal
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in greater_equal
/home/michel/.virtualenvs/rome_naf/lib/python3.5/site-packages/ipykernel/__main__.py:97: RuntimeWarning: invalid value encountered in greater_equal


In [14]:
with open('suggestions.pickle', 'wb') as f:
    pickle.dump(suggestion_array, f)